In [ ]:
%pip install ultralytics roboflow easyocr

In [ ]:
from roboflow import Roboflow
from google.colab import userdata
rf = Roboflow(api_key=userdata.get('ROBOFLOW_API_KEY'))
project = rf.workspace("ivan-fbups").project("license-plate-malaysia-kqy48-looza")
version = project.version(2)
dataset = version.download("yolov8")


## Import depencencies


In [ ]:
import os
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data='/content/License-Plate-(Malaysia)-2/data.yaml' epochs=100 imgsz=512 plots=True project='/content/drive/MyDrive/CarPlateProject',

# Test OCR

In [ ]:
import cv2
import easyocr
import matplotlib.pyplot as plt

# 1. Load your trained model
model = YOLO(f'/content/drive/MyDrive/CarPlateProject/train/weights/best.pt')

# 2. Initialize EasyOCR (English is standard for Msia plates)
reader = easyocr.Reader(['en'])

# 3. Run Inference
image_path = "/content/drive/MyDrive/CarPlateProject/carplate1.jpeg" # Replace with your image
results = model(image_path)

import cv2
import numpy as np
import easyocr
import matplotlib.pyplot as plt

# ... (Load model and reader as before) ...

# 4. Process Results
img = cv2.imread(image_path)
for result in results:
    boxes = result.boxes.cpu().numpy()
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # --- FIX 1: Add Padding (Expand the box by 5-10 pixels) ---
        h, w, _ = img.shape
        pad = 10
        # Ensure we don't go outside the image boundaries
        y1_pad = max(0, y1 - pad)
        y2_pad = min(h, y2 + pad)
        x1_pad = max(0, x1 - pad)
        x2_pad = min(w, x2 + pad)

        plate_crop = img[y1_pad:y2_pad, x1_pad:x2_pad]

        # --- FIX 2: Better Preprocessing ---
        # Convert to grayscale
        gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Upscale the image (EasyOCR works better on larger text)
        gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

        # Use Adaptive Thresholding (Handles shadows/glare better than simple threshold)
        # This keeps the characters black and background white (or vice versa) more reliably
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)

        # --- DEBUG: See what the OCR actually sees ---
        plt.figure()
        plt.title("What OCR sees")
        plt.imshow(thresh, cmap='gray')
        plt.show()

        # Read text
        ocr_result = reader.readtext(thresh)

        # Display Result
        for (bbox, text, prob) in ocr_result:
            print(f"Detected Text: {text} (Confidence: {prob:.2f})")

# Show original image with box
plt.figure()
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

### Yolo is successfully identified the carplate
### However, EasyOCR have very bad performance. Hence we change to PaddleOCR